In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
%cd sample_data

/content/sample_data


In [ ]:
credit_card_data=pd.read_csv('fraudTrain.csv')

In [ ]:
credit_card_data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0.0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0.0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0.0


In [ ]:
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174716 entries, 0 to 174715
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             174716 non-null  int64  
 1   trans_date_trans_time  174716 non-null  object 
 2   cc_num                 174716 non-null  int64  
 3   merchant               174716 non-null  object 
 4   category               174716 non-null  object 
 5   amt                    174716 non-null  float64
 6   first                  174716 non-null  object 
 7   last                   174716 non-null  object 
 8   gender                 174716 non-null  object 
 9   street                 174716 non-null  object 
 10  city                   174716 non-null  object 
 11  state                  174716 non-null  object 
 12  zip                    174716 non-null  int64  
 13  lat                    174716 non-null  float64
 14  long                   174716 non-nu

In [ ]:
credit_card_data.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               1
is_fraud                 1
dtype: int64

In [ ]:
credit_card_data.dropna(subset=['merch_long', 'is_fraud'], inplace=True)

In [ ]:
credit_card_data['is_fraud'].value_counts()

is_fraud
0.0    173195
1.0      1520
Name: count, dtype: int64

Here we see there is an unbalanced data, the number of legit transactions>>>no.of frauds

0 --> Normal Transaction

1 --> fraudulent transaction

In [ ]:
legit = credit_card_data[credit_card_data.is_fraud == 0]
fraud = credit_card_data[credit_card_data.is_fraud == 1]

In [ ]:
print(legit.shape)
print(fraud.shape)

(173195, 23)
(1520, 23)


In [ ]:
legit.amt.describe()

count    173195.000000
mean         67.667648
std         153.237217
min           1.000000
25%           9.590000
50%          47.450000
75%          82.760000
max       15034.180000
Name: amt, dtype: float64

In [ ]:
fraud.amt.describe()

count    1520.000000
mean      509.019586
std       398.552525
min         4.500000
25%        95.152500
50%       343.570000
75%       899.040000
max      1334.070000
Name: amt, dtype: float64

In [ ]:
numeric_df = credit_card_data.select_dtypes(include=['number'])

The dataframe also contains objects. As we are finding out the mean , we will not be including those. We only work will numeric values

In [ ]:
numeric_df.groupby('is_fraud').mean()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long
is_fraud,,,,,,,,,,
0,87411.776408,4.174095e+17,67.667648,48797.671630,38.532869,-90.208382,90091.823436,1.329616e+09,38.534916,-90.208715
1,81115.552632,3.770880e+17,509.019586,48541.086842,39.384581,-90.585600,113519.406579,1.329338e+09,39.379935,-90.589207


Under-Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions --> 1520



In [ ]:
legit_sample_train = legit.sample(n=1520)

In [ ]:
new_dataset_train = pd.concat([legit_sample_train, fraud], axis=0)

In [ ]:
new_dataset_train.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
45142,45142,2019-01-27 21:11:26,6011504998544485,fraud_Zulauf LLC,personal_care,66.62,Ashley,Whitney,F,4038 Smith Avenue,...,32.5104,-86.8138,1089,Materials engineer,1971-11-02,851ad30b0a9e46c2ea332d91121a3acf,1327698686,32.040504,-85.817554,0.0
129272,129272,2019-03-12 04:36:47,180046165512893,"fraud_Bahringer, Osinski and Block",food_dining,131.47,Erica,Walker,F,084 Holmes Avenue Suite 206,...,44.8021,-122.7624,9587,"Engineer, biomedical",1968-03-24,5d52b4e75a184fbad4ec70deb89b4213,1331527007,44.828019,-122.405696,0.0
89959,89959,2019-02-22 23:24:03,3533012926413100,"fraud_Larkin, Stracke and Greenfelder",entertainment,5.08,Megan,Bernard,F,128 Nathan Ramp,...,43.2237,-75.1612,1684,IT trainer,1994-03-01,6599293cf02f6b26458a1a7a30e00f2b,1329953043,43.067333,-75.207381,0.0
111176,111176,2019-03-04 11:54:05,30181367542315,fraud_Kris-Weimann,misc_net,3.10,Danielle,Walker,F,9824 Mitchell Run,...,44.2415,-121.0113,2016,Local government officer,1953-12-25,4fdcb7a512f06ff6e36e68a3ae5fdd7c,1330862045,44.716241,-121.399466,0.0
333,333,2019-01-01 04:22:26,3543299015720986,fraud_Hermann-Gaylord,misc_pos,56.98,Angela,White,F,137 Adam Dale,...,36.8809,-82.0956,9521,"Teacher, English as a foreign language",1977-10-19,5fc9b5a9a63a8fc85e6ddc311d7545e0,1325391746,37.867296,-81.335353,0.0


Doing the same for test dataset

In [ ]:
credit_card_data_test=pd.read_csv('fraudTest.csv')

In [ ]:
credit_card_data_test.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497.0,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1.371817e+09,33.986391,-81.200714,0.0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302.0,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1.371817e+09,39.450498,-109.960431,0.0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496.0,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1.371817e+09,40.495810,-74.196111,0.0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767.0,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1.371817e+09,28.812398,-80.883061,0.0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126.0,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1.371817e+09,44.959148,-85.884734,0.0


In [ ]:
credit_card_data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178555 entries, 0 to 178554
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             178555 non-null  int64  
 1   trans_date_trans_time  178555 non-null  object 
 2   cc_num                 178555 non-null  int64  
 3   merchant               178555 non-null  object 
 4   category               178555 non-null  object 
 5   amt                    178555 non-null  float64
 6   first                  178555 non-null  object 
 7   last                   178555 non-null  object 
 8   gender                 178555 non-null  object 
 9   street                 178555 non-null  object 
 10  city                   178555 non-null  object 
 11  state                  178554 non-null  object 
 12  zip                    178554 non-null  float64
 13  lat                    178554 non-null  float64
 14  long                   178554 non-nu

In [ ]:
credit_card_data_test.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    1
zip                      1
lat                      1
long                     1
city_pop                 1
job                      1
dob                      1
trans_num                1
unix_time                1
merch_lat                1
merch_long               1
is_fraud                 1
dtype: int64

In [ ]:
credit_card_data.dropna(subset=['merch_long', 'is_fraud','merch_lat','unix_time','trans_num','dob','job','city_pop','long','lat','zip','state'], inplace=True)

In [ ]:
credit_card_data_test['is_fraud'].value_counts()

is_fraud
0.0    177795
1.0       759
Name: count, dtype: int64

In [ ]:
legit_test = credit_card_data[credit_card_data.is_fraud == 0]
fraud_test = credit_card_data[credit_card_data.is_fraud == 1]

In [ ]:
numeric_df_test = credit_card_data_test.select_dtypes(include=['number'])

In [ ]:
numeric_df_test.groupby('is_fraud').mean()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long
is_fraud,,,,,,,,,,
0.0,89265.554673,4.189986e+17,67.443384,48830.810895,38.547011,-90.217598,88711.182739,1.374489e+09,38.545720,-90.216216
1.0,91840.432148,5.954605e+17,498.964493,47147.289855,38.892735,-90.278716,45545.231884,1.374565e+09,38.874515,-90.292089


In [ ]:
legit_sample_test = legit.sample(n=759)

In [ ]:
new_dataset_test = pd.concat([legit_sample_test, fraud], axis=0)

In [ ]:
new_dataset_test.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
167433,167433,2019-03-29 23:45:50,4855488158131690372,fraud_Schoen Ltd,kids_pets,56.67,Jeremy,Roberson,M,25887 Martin Inlet,...,44.1554,-72.4847,970,Agricultural consultant,1993-09-29,7fe32549685e367afda25c1fb11e1ac4,1333064750,44.039397,-72.802644,0.0
42290,42290,2019-01-26 05:27:29,4119762878330989,fraud_Kling-Ernser,entertainment,41.58,Melinda,Gutierrez,F,1497 West Gateway,...,40.7152,-89.2706,6713,Archivist,1945-08-19,2944f0d9a00004056bcf69275a74d4d1,1327555649,41.064458,-88.498799,0.0
34121,34121,2019-01-21 06:08:10,2291163933867244,fraud_Heaney-Marquardt,entertainment,23.35,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,d0857fb3c7fd0b85ab4a2f2e4543d939,1327126090,33.801429,-80.998221,0.0
156008,156008,2019-03-24 14:36:15,30541100351706,"fraud_Goyette, Howell and Collier",shopping_pos,1.53,Derrick,George,M,575 Chapman Union,...,30.0377,-95.5326,316665,"Education officer, museum",1990-09-12,0f75b6505e71f6aae2141eb7fcc1f93e,1332599775,30.801344,-95.034785,0.0
85929,85929,2019-02-19 21:46:05,345933964507467,"fraud_Berge, Kautzer and Harris",personal_care,201.06,Carol,Dillon,F,27479 Reeves Dale,...,38.4121,-75.2811,718,Regulatory affairs officer,1985-03-19,f73d8eb60b9d3e0234d6947ee0db8621,1329687965,37.755565,-75.682414,0.0


In [ ]:
new_dataset_test['is_fraud'].value_counts()

is_fraud
1.0    1520
0.0     759
Name: count, dtype: int64

In [ ]:
numeric_newdataset_test = new_dataset_test.select_dtypes(include=['number'])

In [ ]:
numeric_newdataset_test.groupby('is_fraud').mean()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long
is_fraud,,,,,,,,,,
0.0,88692.370224,5.292227e+17,66.903702,49222.916996,38.315193,-90.080634,81059.252964,1.329671e+09,38.318177,-90.033366
1.0,81115.552632,3.770880e+17,509.019586,48541.086842,39.384581,-90.585600,113519.406579,1.329338e+09,39.379935,-90.589207


Splitting train dataset into features and targets

In [ ]:
new_dataset_train['trans_date_trans_time'] = pd.to_datetime(new_dataset_train['trans_date_trans_time'])


In [ ]:
new_dataset_train['trans_year'] = new_dataset_train['trans_date_trans_time'].dt.year
new_dataset_train['trans_month'] = new_dataset_train['trans_date_trans_time'].dt.month
new_dataset_train['trans_day'] = new_dataset_train['trans_date_trans_time'].dt.day
new_dataset_train['trans_hour'] =new_dataset_train['trans_date_trans_time'].dt.hour
new_dataset_train['trans_minute'] = new_dataset_train['trans_date_trans_time'].dt.minute
new_dataset_train['trans_second'] = new_dataset_train['trans_date_trans_time'].dt.second
new_dataset_train['trans_dayofweek'] = new_dataset_train['trans_date_trans_time'].dt.dayofweek

In [ ]:


categorical_columns = new_dataset_train.select_dtypes(include=['object']).columns


In [ ]:
for column in categorical_columns:
        new_dataset_train[column].fillna(new_dataset_train[column].mode()[0], inplace=True)


In [ ]:
new_dataset_train = pd.get_dummies(new_dataset_train, drop_first=True)

In [ ]:
X = new_dataset_train.drop(columns='is_fraud', axis=1)
Y = new_dataset_train['is_fraud']

In [ ]:
print(X)

        Unnamed: 0            cc_num     amt    zip      lat      long  \
45142        45142  6011504998544485   66.62  36749  32.5104  -86.8138   
129272      129272   180046165512893  131.47  97383  44.8021 -122.7624   
89959        89959  3533012926413100    5.08  13304  43.2237  -75.1612   
111176      111176    30181367542315    3.10  97753  44.2415 -121.0113   
333            333  3543299015720986   56.98  24266  36.8809  -82.0956   
...            ...               ...     ...    ...      ...       ...   
173281      173281  4629451965224809  623.89  31632  30.9416  -83.3574   
173324      173324  6597888193422452  875.12  76366  33.6751  -98.6576   
173421      173421  3566094707272327   98.70  91321  34.3795 -118.5230   
173579      173579  6597888193422452   13.53  76366  33.6751  -98.6576   
173701      173701  3566094707272327  128.49  91321  34.3795 -118.5230   

        city_pop  merch_lat  merch_long  trans_year  ...  \
45142       1089  32.040504  -85.817554        2019

In [ ]:
print(Y)

45142     0.0
129272    0.0
89959     0.0
111176    0.0
333       0.0
         ... 
173281    1.0
173324    1.0
173421    1.0
173579    1.0
173701    1.0
Name: is_fraud, Length: 3040, dtype: float64


Comparing Accuracy using Different models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X, Y)

LogisticRegression()

In [ ]:
X_test = numeric_newdataset_test.drop('is_fraud', axis=1, errors='ignore')  # Use errors='ignore' if 'is_fraud' is not in test set


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
common_features_list = list(common_features)
X_train = X[common_features_list]
X_test = X_test[common_features_list]

# Apply the same data preprocessing steps to both the training and test data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the model using the scaled training data
model = LogisticRegression()
model.fit(X_train_scaled, Y)

# Make predictions on the scaled test data
y_pred = model.predict(X_test_scaled)

In [ ]:

y_test = numeric_newdataset_test['is_fraud']


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

Accuracy: 0.8007898200965335, Precision: 0.9634782608695652, Recall: 0.7289473684210527


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [ ]:
model.fit(X_train_scaled, Y)

RandomForestClassifier()

In [ ]:
y_pred = model.predict(X_test_scaled)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

Accuracy: 0.9741114523913997, Precision: 0.9626345788473718, Recall: 1.0
